In [10]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [11]:
categories = ["1991","2","26","21","1987","165"]

hrefs = []

In [12]:

cookies = {
    'quform_session_4afe269b4eb47475f1c7f9adf026786c': 'xYvOSPfIFUzqo6L6upzOs4ayvg1eXwdyEeWH6Sx9',
    '_gcl_au': '1.1.957906330.1689463467',
    'AMCVS_43D8021954C26BE10A4C98A5%40AdobeOrg': '1',
    '_cc_id': '3eda540b1733bbf97d5a84667177a178',
    'izootoWpConfig': '%7B%22b_type%22:7,%22d_type%22:1,%22evt_trk%22:1,%22izooto_uid%22:%229632b6a4-e80a-4536-8377-4839edcc023e%22%7D',
    'AMCV_43D8021954C26BE10A4C98A5%40AdobeOrg': '-1124106680%7CMCIDTS%7C19560%7CMCMID%7C18653459563264821612797502131604271316%7CMCAAMLH-1690540353%7C3%7CMCAAMB-1690540353%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1689942753s%7CNONE%7CMCSYNCS%7C1083-19561*1085-19561*1086-19561*1087-19561*1088-19561*19913-19561*83349-19561%7CMCSYNCSOP%7C411-19561%7CvVersion%7C5.2.0',
    '_gid': 'GA1.3.1939725000.1689935554',
    'panoramaId_expiry': '1690021954247',
    'panoramaId': '24aa7eb4ab23dee1ace84787366ea9fb927af0baabde5e68c1e8fb79cc4932a0',
    'panoramaIdType': 'panoDevice',
    '_dc_gtm_UA-6654833-14': '1',
    '_gat_UA-20278225-104': '1',
    '_gat_UA-6654833-14': '1',
    '_ga_3JY79DYEG4': 'GS1.1.1690009540.4.1.1690010902.0.0.0',
    '_ga_GV9EQDZN1L': 'GS1.1.1690009540.4.1.1690010902.0.0.0',
    '_ga': 'GA1.3.1199758714.1689463467',
    '_ga_17VJVCH5M2': 'GS1.3.1690009541.3.1.1690010902.42.0.0',
    'FCNEC': '%5B%5B%22AKsRol_5qyJrdF-lAcxUXNHqqSoKlcBbPaxgKF02Xo9Nn5UZHfpIs3qIgrkiXd_XZwCQtiPh9VolJIrn9MWDuF4xpE_C8n_v02p2gpDPheu8s0KgbFFoXalyRczqG2kmDEAhYt3AXY6HaMDt0GMZSfm0Eu3DSdKM1g%3D%3D%22%5D%2Cnull%2C%5B%5D%5D',
}

headers = {
    'authority': 'www.umpan.com.my',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    # 'cookie': 'quform_session_4afe269b4eb47475f1c7f9adf026786c=xYvOSPfIFUzqo6L6upzOs4ayvg1eXwdyEeWH6Sx9; _gcl_au=1.1.957906330.1689463467; AMCVS_43D8021954C26BE10A4C98A5%40AdobeOrg=1; _cc_id=3eda540b1733bbf97d5a84667177a178; izootoWpConfig=%7B%22b_type%22:7,%22d_type%22:1,%22evt_trk%22:1,%22izooto_uid%22:%229632b6a4-e80a-4536-8377-4839edcc023e%22%7D; AMCV_43D8021954C26BE10A4C98A5%40AdobeOrg=-1124106680%7CMCIDTS%7C19560%7CMCMID%7C18653459563264821612797502131604271316%7CMCAAMLH-1690540353%7C3%7CMCAAMB-1690540353%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1689942753s%7CNONE%7CMCSYNCS%7C1083-19561*1085-19561*1086-19561*1087-19561*1088-19561*19913-19561*83349-19561%7CMCSYNCSOP%7C411-19561%7CvVersion%7C5.2.0; _gid=GA1.3.1939725000.1689935554; panoramaId_expiry=1690021954247; panoramaId=24aa7eb4ab23dee1ace84787366ea9fb927af0baabde5e68c1e8fb79cc4932a0; panoramaIdType=panoDevice; _dc_gtm_UA-6654833-14=1; _gat_UA-20278225-104=1; _gat_UA-6654833-14=1; _ga_3JY79DYEG4=GS1.1.1690009540.4.1.1690010902.0.0.0; _ga_GV9EQDZN1L=GS1.1.1690009540.4.1.1690010902.0.0.0; _ga=GA1.3.1199758714.1689463467; _ga_17VJVCH5M2=GS1.3.1690009541.3.1.1690010902.42.0.0; FCNEC=%5B%5B%22AKsRol_5qyJrdF-lAcxUXNHqqSoKlcBbPaxgKF02Xo9Nn5UZHfpIs3qIgrkiXd_XZwCQtiPh9VolJIrn9MWDuF4xpE_C8n_v02p2gpDPheu8s0KgbFFoXalyRczqG2kmDEAhYt3AXY6HaMDt0GMZSfm0Eu3DSdKM1g%3D%3D%22%5D%2Cnull%2C%5B%5D%5D',
    'origin': 'https://www.umpan.com.my',
    'referer': 'https://www.umpan.com.my/category/umpanpedia/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    'td_theme_name': 'Newspaper',
    'v': '12.4',
}




In [13]:
def crawl(url,category):
    
        
    data = {
    'action': 'td_ajax_loop',
    'loopState[sidebarPosition]': '',
    'loopState[moduleId]': '1',
    'loopState[currentPage]': f'{url}',
    'loopState[atts][category_id]': f'{category}',
    'loopState[ajax_pagination_infinite_stop]': '3',
    'loopState[server_reply_html_data]': '',
    }
    
    while True:
        try:
            response = requests.post('https://www.umpan.com.my/wp-admin/admin-ajax.php', params=params, cookies=cookies, headers=headers, data=data)
            if response.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)
    
    try:
        r = response.json()
    except:
        return
    
    
    soup = BeautifulSoup(r['server_reply_html_data'], "lxml")
    
    block = soup.find_all("h3",attrs = {"class":"entry-title td-module-title"})
    
    if block is None:
        return

    for link in soup.find_all("h3",attrs = {"class":"entry-title td-module-title"}):

        href = link.find('a').get('href')
        hrefs.append(href)
    

In [14]:
max_worker = 5
hrefs = []

for index, c in enumerate(categories):
    max_pages = ['214', '99', '48', '90', '13',"87"]
    max_page = int(max_pages[index])
    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, min(i + max_worker, max_page + 1))  # Fix range issue
        urls = [str(a) for a in aranged]  # Use the correct range of pages
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url, c): url for url in urls}
            
        for future in as_completed(futures):
            future.result()

100%|█| 43/43 [00:55<00:00,
100%|█| 20/20 [00:25<00:00,
100%|█| 10/10 [00:09<00:00,
100%|█| 18/18 [00:20<00:00,
100%|█| 3/3 [00:03<00:00,  
100%|█| 18/18 [00:18<00:00,


In [17]:
len(hrefs)

5477

In [16]:
hrefs = list(set(hrefs))

In [18]:
with open('umpan-link.json', 'a') as f:
    json.dump(hrefs, f)

In [4]:
with open('umpan-link.json') as f:
    hrefs = json.load(f)

In [20]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title").text
        h = soup.find('div', attrs = {"class":"td-ss-main-content"})
        p = [p.text for p in h.find_all("p") if len(p.text) > 3 and not p.find('a', href=True)]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None
    
    time.sleep(3)

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [21]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('umpan.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')

 60%|▌| 3281/5477 [51:16<40

("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))


100%|█| 5477/5477 [1:25:39<
